In [6]:
import os

from datetime import datetime, timedelta
import json
import requests

In [7]:
DATE_FORMAT = "%Y-%m-%d"
WORK_PROJECTS = [186181594, 188079427]
TOOLING_PROJECTS = [187944113,186181587,188328043]

API_KEY = os.environ.get('TOGGL_API_KEY')

SUMMARY_URL = "https://api.track.toggl.com/reports/api/v3/workspace/397836/summary/time_entries"
CURRENT_ENTRY_URL = "https://api.track.toggl.com/api/v9/me/time_entries/current"

In [11]:
def get_entry_secs_from_toggl(start_date, end_date, project_ids):
    time_entries = requests.post(
        SUMMARY_URL,
        headers={"Content-Type": "application/json"},
        data=json.dumps(
            {"end_date": end_date, "start_date": start_date, "project_ids": project_ids}
        ),
        auth=(API_KEY, "api_token"),
    ).json()

    seconds = 0
    groups = time_entries["groups"]
    for group in groups:
        for sub_group in group["sub_groups"]:
            seconds += sub_group["seconds"]
    return seconds

def get_current_time_entry_secs_from_toggl(project_ids):
    time_entry = requests.get(
        CURRENT_ENTRY_URL,
        headers={"Content-Type": "application/json"},
        auth=(API_KEY, "api_token"),
    ).json()

    if 'start' in time_entry and 'project_id' in time_entry and time_entry['project_id'] in project_ids:
        cts = datetime.strptime(time_entry['start'], '%Y-%m-%dT%H:%M:%S%z')
        diff = datetime.now(cts.tzinfo) - cts
        return diff.seconds

    return 0

def get_time_entry_secs(project_ids, end_date, duration):
    end_date_str = end_date.strftime(DATE_FORMAT)
    start_date = end_date - timedelta(days=(duration - 1))
    start_date_str = start_date.strftime(DATE_FORMAT)
    today_secs = get_entry_secs_from_toggl(start_date_str, end_date_str, project_ids)
    if end_date == datetime.today():
        today_secs += get_current_time_entry_secs_from_toggl(project_ids)
    return today_secs



def get_today_time(project_ids):
    return get_time_entry_secs(project_ids, datetime.today(), 1)

def get_week_time(project_ids):
    return get_time_entry_secs(project_ids, datetime.today(), 7)


In [26]:
total_mins_today = minutes + summary_today/60
total_mins_last_week = minutes + summary_last_week/60

hours_today = total_mins_today/60
mod_mins_today = total_mins_today%60
pad with leading zero if needed
if mod_mins_today < 10:
    mod_mins_today = "0" + str(mod_mins_today)

open("/Users/danthompson/Code/Scripts/CLI/toggl textbar/data/opt_tooling_time_today.txt", "w").write(str(total_mins_today))
open("/Users/danthompson/Code/Scripts/CLI/toggl textbar/data/opt_tooling_time_last_week.txt", "w").write(str(total_mins_last_week))

print "%s:%s" % (hours_today, mod_mins_today)


10670

In [9]:
get_current_time_entry_from_toggl(WORK_PROJECTS)

{'id': 2821607554,
 'workspace_id': 397836,
 'project_id': 186181587,
 'task_id': None,
 'billable': False,
 'start': '2023-01-29T23:19:10+00:00',
 'stop': None,
 'duration': -1675034350,
 'description': 'Worthwhile',
 'tags': None,
 'tag_ids': None,
 'duronly': True,
 'at': '2023-01-29T23:19:10+00:00',
 'server_deleted_at': None,
 'user_id': 522161,
 'uid': 522161,
 'wid': 397836,
 'pid': 186181587}

In [40]:
summary_today = get_summaries_from_toggl(today, today, [186181594, 188079427])
summary_last_week = get_summaries_from_toggl(last_week, today, [186181594, 188079427])

total = get_seconds_from_summary(summary_today)
total += get_seconds_from_summary(summary_last_week)


In [57]:
groups = (group['sub_groups'] for group in summary_last_week['groups'])
seconds = (subgroup['seconds'] for group in groups for subgroup in group)
print(list(seconds))

[2400, 1800, 15500, 6919, 10234, 5564, 3182, 7635]


In [36]:
summary_today

[{'id': 186181594,
  'sub_groups': [{'id': None, 'title': 'Business ideas', 'seconds': 2400},
   {'id': None, 'title': 'Course', 'seconds': 840},
   {'id': None, 'title': 'Course - Chp 9', 'seconds': 2050}]}]

In [37]:
def get_seconds_from_summary(summary_json):
    seconds = 0
    groups = summary_json["groups"]
    for group in groups:
        for sub_group in group["sub_groups"]:
            seconds += sub_group["seconds"]
    return seconds

In [38]:
get_seconds_from_summary(summary_today)

5290